<a href="https://colab.research.google.com/github/SankaviKanesalingam/mlproject/blob/main/npy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---

# <center>★ ML Project - Object Detection ★
#### <center> ***Domain : Computer Vision***

---

<center> <img src="https://raw.githubusercontent.com/Masterx-AI/Project_Object_Detection_Yolo_V4/main/result.jpg" style="width: 600px;"/> <center/>

---

### Description:

OpenCV is the huge open-source library for computer vision, machine learning, and image processing and now it plays a major role in real-time operation which is very important in today’s systems. By using it, one can process images and videos to identify objects, faces, or even the handwriting of a human. This article focuses on detecting objects.

Object Detection is a computer technology related to computer vision, image processing, and deep learning that deals with detecting instances of objects in images and videos. We will do object detection in this article using something known as haar cascades.

YOLOv4 is a one-stage object detection model that improves on YOLOv3 with several bags of tricks and modules introduced in the literature. The components section below details the tricks and modules used.


### Acknowledgement:
OpenCV Library developed by Intel

### Objective:
- Object Detection with OpenCV using Yolo V4

---

# <center> 1. Data Exploration

In [ ]:
import numpy as np
import cv2 # openCV
import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"]=[12,8]

# check the opencv version
print(cv2.__version__)

In [ ]:
import requests
url = 'https://raw.githubusercontent.com/Masterx-AI/Project_Object_Detection_Yolo_V4/main/people_bicycles.jpg'
r = requests.get(url, allow_redirects=True)
open('people_bicycles.jpg', 'wb').write(r.content)

In [ ]:
# Reading the image file
test_img = cv2.imread('people_bicycles.jpg')
img = cv2.cvtColor(test_img, cv2.COLOR_BGR2RGB)

# Plotting the image
def plot_image(img, cmap=None):
    plt.imshow(img, cmap=cmap)
    plt.xticks([])
    plt.yticks([])

plot_image(img)
plt.show()

---

# <center> 2. Image Processing

In [ ]:
# first create a directory to store the model
%mkdir model

# enter the directory and download the necessary files
%cd model
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.weights
!wget https://raw.githubusercontent.com/AlexeyAB/darknet/master/cfg/yolov4.cfg
!wget https://raw.githubusercontent.com/AlexeyAB/darknet/master/data/coco.names
%cd ..

In [ ]:
# Converting to blob object

scalefactor = 1.0/255.0
new_size = (416, 416)
blob = cv2.dnn.blobFromImage(test_img, scalefactor, new_size, swapRB=True, crop=False)

In [ ]:
# define class labels
class_labels_path = "model/coco.names"
class_labels = open(class_labels_path).read().strip().split("\n")
class_labels

# declare repeating bounding box colors for each class
# 1st: create a list colors as an RGB string array
# Example: Red, Green, Blue, Yellow, Magenda
class_colors = ["255,0,0","0,255,0","0,0,255","255,155,0","255,0, 255"]

#2nd: split the array on comma-separated strings and for change each string type to integer
class_colors = [np.array(every_color.split(",")).astype("int") for every_color in class_colors]

#3rd: convert the array or arrays to a numpy array
class_colors = np.array(class_colors)

#4th: tile this to get 80 class colors, i.e. as many as the classes(16 rows of 5cols each).
# If you want unique colors for each class you may randomize the color generation or set them manually
class_colors = np.tile(class_colors,(16,1))

def colored(r, g, b, text):
    return "\033[38;2;{};{};{}m{} \033[38;2;255;255;255m".format(r, g, b, text)

for i in range(16):
    line = ""
    for j in range(5):
        class_id = i*5 + j
        class_id_str = str(class_id)
        text = "class" + class_id_str
        colored_text = colored(class_colors[class_id][0], class_colors[class_id][1], class_colors[class_id][2], text)
        line += colored_text
    print(line)

# or select the colors randomly
class_colors = np.random.randint(0, 255, size=(len(class_labels), 3), dtype="uint8")

---

# <center>  3. Object Detection

In [ ]:
# Load the pre-trained model
yolo_model = cv2.dnn.readNetFromDarknet('model/yolov4.cfg','model/yolov4.weights')

# Read the network layers/components. The YOLO V4 neural network has 379 components. They consist of convolutional layers (conv), rectifier linear units (relu) etc.:
model_layers = yolo_model.getLayerNames()
print("number of network components: " + str(len(model_layers)))
# print(model_layers)

# extract the output layers in the code that follows:
# - model_layer[0]: returns the index of each output layer in the range of 1 to 379
# - model_layer[0] - 1: corrects  this to the range of 0 to 378
# - model_layers[model_layer[0] - 1]: returns the indexed layer name
output_layers = [model_layers[model_layer - 1] for model_layer in yolo_model.getUnconnectedOutLayers()]

# YOLOv4 deploys the same YOLO head as YOLOv3 for detection with the anchor based detection steps, and three levels of detection granularity.
print(output_layers)

In [ ]:
# input pre-processed blob into the model
yolo_model.setInput(blob)

# compute the forward pass for the input, storing the results per output layer in a list
obj_detections_in_layers = yolo_model.forward(output_layers)

# verify the number of sets of detections
print("number of sets of detections: " + str(len(obj_detections_in_layers)))

In [ ]:
def object_detection_analysis(test_image, obj_detections_in_layers, confidence_threshold):

  # get the image dimensions
  img_height = test_img.shape[0]
  img_width = test_img.shape[1]

  result = test_image.copy()

  # loop over each output layer
  for object_detections_in_single_layer in obj_detections_in_layers:
    # loop over the detections in each layer
      for object_detection in object_detections_in_single_layer:
        # obj_detection[1]: bbox center pt_x
        # obj_detection[2]: bbox center pt_y
        # obj_detection[3]: bbox width
        # obj_detection[4]: bbox height
        # obj_detection[5]: confidence scores for all detections within the bbox

        # get the confidence scores of all objects detected with the bounding box
        prediction_scores = object_detection[5:]
        # consider the highest score being associated with the winning class
        # get the class ID from the index of the highest score
        predicted_class_id = np.argmax(prediction_scores)
        # get the prediction confidence
        prediction_confidence = prediction_scores[predicted_class_id]

        # consider object detections with confidence score higher than threshold
        if prediction_confidence > confidence_threshold:
            # get the predicted label
            predicted_class_label = class_labels[predicted_class_id]
            # compute the bounding box coordinates scaled for the input image
            # scaling is a multiplication of the float coordinate with the appropriate  image dimension
            bounding_box = object_detection[0:4] * np.array([img_width, img_height, img_width, img_height])
            # get the bounding box centroid (x,y), width and height as integers
            (box_center_x_pt, box_center_y_pt, box_width, box_height) = bounding_box.astype("int")
            # to get the start x and y coordinates we to subtract from the centroid half the width and half the height respectively
            # for even values of width and height of bboxes adjacent to the  image border
            #  this may generate a -1 which is prevented by the max() operator below
            start_x_pt = max(0, int(box_center_x_pt - (box_width / 2)))
            start_y_pt = max(0, int(box_center_y_pt - (box_height / 2)))
            end_x_pt = start_x_pt + box_width
            end_y_pt = start_y_pt + box_height

            # get a random mask color from the numpy array of colors
            box_color = class_colors[predicted_class_id]

            # convert the color numpy array as a list and apply to text and box
            box_color = [int(c) for c in box_color]

            # print the prediction in console
            predicted_class_label = "{}: {:.2f}%".format(predicted_class_label, prediction_confidence * 100)
            print("predicted object {}".format(predicted_class_label))

            # draw the rectangle and text in the image
            cv2.rectangle(result, (start_x_pt, start_y_pt), (end_x_pt, end_y_pt), box_color, 1)
            cv2.putText(result, predicted_class_label, (start_x_pt, start_y_pt-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, box_color, 1)
  return result

confidence_threshold = 0.2
result_raw = object_detection_analysis(test_img, obj_detections_in_layers, confidence_threshold)

result_img = cv2.cvtColor(result_raw, cv2.COLOR_BGR2RGB)
plt.imshow(result_img)
plt.show()

In [ ]:
class_ids_list = []
boxes_list = []
confidences_list = []

In [ ]:
def object_detection_attributes(test_image, obj_detections_in_layers, confidence_threshold):
  # get the image dimensions
  img_height = test_img.shape[0]
  img_width = test_img.shape[1]

  # loop over each output layer
  for object_detections_in_single_layer in obj_detections_in_layers:
    # loop over the detections in each layer
    for object_detection in object_detections_in_single_layer:
      # get the confidence scores of all objects detected with the bounding box
      prediction_scores = object_detection[5:]
      # consider the highest score being associated with the winning class
      # get the class ID from the index of the highest score
      predicted_class_id = np.argmax(prediction_scores)
      # get the prediction confidence
      prediction_confidence = prediction_scores[predicted_class_id]

      # consider object detections with confidence score higher than threshold
      if prediction_confidence > confidence_threshold:
        # get the predicted label
        predicted_class_label = class_labels[predicted_class_id]
        # compute the bounding box coordinates scaled for the input image
        bounding_box = object_detection[0:4] * np.array([img_width, img_height, img_width, img_height])
        (box_center_x_pt, box_center_y_pt, box_width, box_height) = bounding_box.astype("int")
        start_x_pt = max(0, int(box_center_x_pt - (box_width / 2)))
        start_y_pt = max(0, int(box_center_y_pt - (box_height / 2)))

        # update the 3 lists for nms processing
        # - confidence is needed as a float
        # - the bbox info has the openCV Rect format
        class_ids_list.append(predicted_class_id)
        confidences_list.append(float(prediction_confidence))
        boxes_list.append([int(start_x_pt), int(start_y_pt), int(box_width), int(box_height)])

In [ ]:
score_threshold = 0.5
object_detection_attributes(test_img, obj_detections_in_layers, score_threshold)

In [ ]:
# NMS for a set of overlapping bboxes returns the ID of the one with highest
# confidence score while suppressing all others (non maxima)
# - score_threshold: a threshold used to filter boxes by score
# - nms_threshold: a threshold used in non maximum suppression.

score_threshold = 0.5
nms_threshold = 0.4
winner_ids = cv2.dnn.NMSBoxes(boxes_list, confidences_list, score_threshold, nms_threshold)

# loop through the final set of detections remaining after NMS and draw bounding box and write text
for winner_id in winner_ids:
    max_class_id = winner_id
    box = boxes_list[max_class_id]
    start_x_pt = box[0]
    start_y_pt = box[1]
    box_width = box[2]
    box_height = box[3]

    #get the predicted class id and label
    predicted_class_id = class_ids_list[max_class_id]
    predicted_class_label = class_labels[predicted_class_id]
    prediction_confidence = confidences_list[max_class_id]

    #obtain the bounding box end coordinates
    end_x_pt = start_x_pt + box_width
    end_y_pt = start_y_pt + box_height

    #get a random mask color from the numpy array of colors
    box_color = class_colors[predicted_class_id]

    #convert the color numpy array as a list and apply to text and box
    box_color = [int(c) for c in box_color]

    # print the prediction in console
    predicted_class_label = "{}: {:.2f}%".format(predicted_class_label, prediction_confidence * 100)
    print("predicted object {}".format(predicted_class_label))

    # draw rectangle and text in the image
    cv2.rectangle(test_img, (start_x_pt, start_y_pt), (end_x_pt, end_y_pt), box_color, 2)
    cv2.putText(test_img, predicted_class_label, (start_x_pt, start_y_pt-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, box_color, 2)

test_imgz = cv2.cvtColor(test_img, cv2.COLOR_BGR2RGB)

plt.imshow(test_imgz)
plt.show()

---

# <center> 4. Video Processing

In [ ]:
# Incase using a youtube video

# import pafy

# url = "https://www.youtube.com/watch?v=eldQI2ONYE0&ab_channel=Angeles69"
# video = pafy.new(url)
# best = video.getbest(preftype="mp4")

In [ ]:
def object_detection_analysis_with_nms(test_img, class_labels, class_colors, obj_detections_in_layers, score_threshold, nms_threshold):

    # get the image dimensions
    img_height = test_img.shape[0]
    img_width = test_img.shape[1]

    result = test_img.copy()

    # declare lists for the arguments of interest: classID, bbox info, detection confidences
    class_ids_list = []
    boxes_list = []
    confidences_list = []
    # loop over each output layer
    for object_detections_in_single_layer in obj_detections_in_layers:
        # loop over the detections in each layer
        for object_detection in object_detections_in_single_layer:
            # get the confidence scores of all objects detected with the bounding box
            prediction_scores = object_detection[5:]
            # consider the highest score being associated with the winning class
            # get the class ID from the index of the highest score
            predicted_class_id = np.argmax(prediction_scores)
            # get the prediction confidence
            prediction_confidence = prediction_scores[predicted_class_id]

            # consider object detections with confidence score higher than threshold
            if prediction_confidence > score_threshold:
                # get the predicted label
                predicted_class_label = class_labels[predicted_class_id]
                # compute the bounding box cooridnates scaled for the input image
                bounding_box = object_detection[0:4] * np.array([img_width, img_height, img_width, img_height])
                (box_center_x_pt, box_center_y_pt, box_width, box_height) = bounding_box.astype("int")
                start_x_pt = max(0, int(box_center_x_pt - (box_width / 2)))
                start_y_pt = max(0, int(box_center_y_pt - (box_height / 2)))

                # update the 3 lists for nms processing
                # - confidence is needed as a float
                # - the bbox info has the openCV Rect format
                class_ids_list.append(predicted_class_id)
                confidences_list.append(float(prediction_confidence))
                boxes_list.append([int(start_x_pt), int(start_y_pt), int(box_width), int(box_height)])

In [ ]:
# Real-time video object detection


new_width = 640
new_height = 480
dim = (new_width, new_height)

cap = cv2.VideoCapture('London.mp4')  #best.url

while cap.isOpened():
    #get the current frame from video stream
    ret,frame = cap.read()

    frame = cv2.resize(frame, dim, interpolation = cv2.INTER_AREA)

    blob = cv2.dnn.blobFromImage(frame, scalefactor, new_size, swapRB=True, crop=False)

    # input pre-processed blob into the model
    yolo_model.setInput(blob)

    # compute the forward pass for the input, storing the results per output layer in a list
    obj_detections_in_layers = yolo_model.forward(output_layers)

#     # get  the object detections drawn on  the frame
    frame, winner_boxes = object_detection_analysis_with_nms(frame, class_labels, class_colors, obj_detections_in_layers, score_threshold, nms_threshold)

    #display the frame
    cv2.imshow('Webcam', frame)
    # if running outside Colab notebooks use:
    # cv2.imshow(frame)

    #terminate while loop if 'q' key is pressed - applicable outside the notebooks
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

#releasing the stream and the camera
cap.release()
# cv2.destroyAllWindows()


---

In [ ]:
#<<<--------------------------------------THE END---------------------------------------->>>